## Mixup 구축

In [88]:
import cv2
from PIL import Image
import glob
import os
def MixUp(img1_path, img2_path, alpha = 0.5):
    """
    image mix up 
    
    Args:
        img1_path : image1 path
        img2_path : image2 path
        
    Returns:
        im_pil : MixUp img(PIL.Image type)
        beta : image1 influence
        alpha : image2 influence
    """
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    
    beta = 1.0 - alpha
    
    
    dst = cv2.addWeighted(img1, beta, img2, alpha, 0)
    img = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
    
    im_pil = Image.fromarray(img)
    return im_pil, beta, alpha

In [89]:
import sys
sys.path.append('/opt/ml/pstage01')
from model import models, loss, metric
from dataloader import mask
from util import meter, transformers, generate

In [90]:
root_dir = "/opt/ml/input/data/"

all_data = sorted(glob.glob(os.path.join(root_dir, 'train', '*', '*', '*')))
img_ex, alpha, beta = MixUp(all_data[6+7*699], all_data[3+7*699], 0.2)
print(img_ex)
img_ex

In [91]:
from PIL import Image
image = Image.open("/opt/ml/input/data/train/images/000001_female_Asian_45/incorrect_mask.jpg")

In [1]:
print(type(image))

In [93]:
from io import BytesIO  
b = BytesIO()
img_ex.save(b, format = "jpeg")
img_ex2 = Image.open(b)
print(type(img_ex2))
img_ex2

In [94]:
root_dir = "/opt/ml/input/data/"

all_data = sorted(glob.glob(os.path.join(root_dir, 'train', '*', '*', '*')))
img_ex, alpha, beta = generate.mix_up(all_data[6+7*699], all_data[3+7*699], 0.5)
img_ex

## cutmix 구현해보기

In [95]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [96]:
root_dir = "/opt/ml/input/data/"

filenames = sorted(glob.glob(os.path.join(root_dir, 'train', '*', '*', '*')))

In [97]:
# image_paths = filenames[:10]

# # Display a sample image
# plt.imshow(cv2.cvtColor(cv2.imread(image_paths[0]), cv2.COLOR_BGR2RGB)); plt.show();

In [1]:
print(type(cv2.imread(image_paths[0]))) # cv2.imread는 이미지 파일을 numpy로 읽는구나

In [99]:
image_batch = []
image_batch_labels = []

n_images = 2

for i in range(n_images):
    image = cv2.cvtColor(cv2.imread(image_paths[i]), cv2.COLOR_BGR2RGB)
    image_batch.append(image)
    
    label_temp = list(np.floor(np.random.rand(1)*2.99).astype(int))[0]
    if label_temp == 0:
        label = [1,0,0]
    elif label_temp == 1:
        label = [0,1,0]
    else: # label_temp == 2
        label = [0,0,1]
    
    image_batch_labels.append(label)

In [101]:
image_batch = np.array(image_batch) # 행렬화 된

In [102]:
image_batch_labels = np.array(image_batch_labels)

In [2]:
image_batch_labels

In [1]:
print()
print(f"Image labels: {image_batch_labels}\n")

In [105]:
for i in range(1):
    for j in range(2):
        plt.subplot(1,2,2*i+j+1)
        plt.imshow(image_batch[2*i+j])
plt.show()

In [106]:
def rand_bbox(size, lamb):
    """ 
    구역 분할해주는 함수
    """
    W = size[0]
    H = size[1]
    cut_rat = np.sqrt(1. - lamb)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [114]:
image = cv2.cvtColor(cv2.imread(image_paths[0]), cv2.COLOR_BGR2RGB)

lamb = 0.3
size = image.shape
print("size : ", size)
bbox = rand_bbox(size, lamb)

print(bbox[0],bbox[1],bbox[2],bbox[3])

# Draw bounding box on the image
im = image.copy()
x1 = bbox[0]
y1 = bbox[1]
x2 = bbox[2]
y2 = bbox[3]
cv2.rectangle(im, (x1, y1), (x2, y2), (255, 0, 0), 3) # 사각형 그리기
plt.imshow(im);
plt.title('Original image with random bounding box')
plt.show();

# Show cropped image
plt.imshow(image[y1:y2, x1:x2]);
plt.title('Cropped image')
plt.show()

In [108]:
def generate_cutmix_image(image_batch, image_batch_labels, beta):
    """
    cutmix generator
    """
    
    # 이거는 예시로 
    lam = np.random.beta(beta, beta)
    rand_index = np.random.permutation(len(image_batch))
    target_a = image_batch_labels
    target_b = image_batch_labels[rand_index]
    bbx1, bby1, bbx2, bby2 = rand_bbox(image_batch[0].shape, lam)
    image_batch_updated = image_batch.copy()
    image_batch_updated[:, bbx1:bbx2, bby1:bby2, :] = image_batch[rand_index, bbx1:bbx2, bby1:bby2, :]
    
    # 비율 다시 맞춰줌
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (image_batch.shape[1] * image_batch.shape[2]))
    
    # lam으로 loss 비율 맞춰줌
    label = target_a * lam + target_b * (1. - lam)  # 라벨 가중치 부여 
    
    
    return image_batch_updated, label
    

In [109]:
input_image = image_batch[0]
image_batch_updated, lam = generate.cut_mix(image_batch, 1.0)

In [112]:
image_batch_updated[0]

In [113]:
image_batch_updated[1]